# Mapping current yarn shops vs population

In [ ]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import numpy as np


# County mapping

In [ ]:
# Read in county shapefile

counties = gpd.read_file('../data/tl_2017_us_county/tl_2017_us_county.shp')
print(counties.crs)
counties.head(2)

In [ ]:
counties.info()

In [ ]:
#take FIPS for territories and drop based on STATEFP column
territories_list = ['60', '66', '69', '72', '78']

counties = counties[~counties['STATEFP'].isin(territories_list)]

fig, ax = plt.subplots(figsize = (20, 20))

counties.plot(column = 'COUNTYNS', ax = ax)

# to tighten the map
plt.xlim(-175, -65);


In [ ]:
counties = counties[['STATEFP', 'COUNTYFP', 'COUNTYNS', 'GEOID', 'NAME', 'geometry']]
counties.head()

# Shop info

In [ ]:
shops_df = pd.read_csv('../data/df_shops_clean2.csv')
shops_df.head(2)


In [ ]:
# create geometry column

shops_df['geometry'] = shops_df.apply(lambda x: Point((float(x.longitude),
                                                       float(x.latitude))),
                                      axis=1)
shops_df.head(3)

In [ ]:
# create geodataframe from shops

shop_geo = gpd.GeoDataFrame(shops_df,
                            crs = counties.crs,
                            geometry = shops_df['geometry'])


In [ ]:
type(shop_geo)

In [ ]:
# combine with spatial join

shops_by_county = gpd.sjoin(shop_geo, counties, op = 'within')
shops_by_county.head()

In [ ]:
shops_by_county.GEOID.value_counts()
# go back to cleaning and check for duplicates

In [ ]:
shops_by_county.info()

In [ ]:
# LA - 36 listings
shops_la = shops_by_county.loc[shops_by_county.GEOID == '06037']

In [ ]:
# Seattle - 29 listings
shops_seattle = shops_by_county.loc[shops_by_county.GEOID == '53033']

In [ ]:
shops_middlesex = shops_by_county.loc[shops_by_county.GEOID == '25017']

In [ ]:
# defining variables for map

polygon_county = counties.loc[counties['GEOID'] == '06037']
center = polygon_county.geometry.centroid
area_center = [center.y, center.x]
shops_county = shops_la

In [ ]:
# mapping shops with folium - LA as example

# create map centered on area
map_county = folium.Map(location = area_center, zoom_start = 9)

# add county polygon to map
folium.GeoJson(polygon_county).add_to(map_county)

# iterate through shops to place points on map, popup marker for shop name
for row_index, row_values in shops_county.iterrows():
    loc = [row_values['latitude'], row_values['longitude']]
    pop = str(row_values['name'])
    marker = folium.Marker(
        location = loc, 
        popup = pop) 
    
    marker.add_to(map_county)
map_county.save('../maps/map_la.html')

map_county

In [ ]:
# mapping shops with folium - King county (Seattle)

# defining variables for map
polygon_county = counties.loc[counties['GEOID'] == '53033']
center = polygon_county.geometry.centroid
area_center = [center.y, center.x]
shops_county = shops_seattle

# create map centered on area
map_county = folium.Map(location = area_center, zoom_start = 9)

# add county polygon to map
folium.GeoJson(polygon_county).add_to(map_county)

# iterate through shops to place points on map, popup marker for shop name
for row_index, row_values in shops_county.iterrows():
    loc = [row_values['latitude'], row_values['longitude']]
    pop = str(row_values['name'])
    marker = folium.Marker(
        location = loc, 
        popup = pop) 
    
    marker.add_to(map_county)
map_county.save('../maps/map_seattle.html')

map_county

In [ ]:
# mapping shops with folium - Middlesex county (part of Boston metro area)

# defining variables for map
polygon_county = counties.loc[counties['GEOID'] == '25017']
center = polygon_county.geometry.centroid
area_center = [center.y, center.x]
shops_county = shops_middlesex

# create map centered on area
map_county = folium.Map(location = area_center, zoom_start = 10)

# add county polygon to map
folium.GeoJson(polygon_county).add_to(map_county)

# iterate through shops to place points on map, popup marker for shop name
for row_index, row_values in shops_county.iterrows():
    loc = [row_values['latitude'], row_values['longitude']]
    pop = str(row_values['name'])
    marker = folium.Marker(
        location = loc, 
        popup = pop) 
    
    marker.add_to(map_county)
map_county.save('../maps/map_middlesex.html')

map_county

In [ ]:
shops_nashville = shops_by_county.loc[shops_by_county.NAME == 'Davidson']

In [ ]:
shops_nashville

In [ ]:
# mapping shops with folium - Davidson County (Nashville)

# defining variables for map
polygon_county = counties.loc[counties['GEOID'] == '47037']
center = polygon_county.geometry.centroid
area_center = [center.y, center.x]
shops_county = shops_nashville

# create map centered on area
map_county = folium.Map(location = area_center, zoom_start = 10)

# add county polygon to map
folium.GeoJson(polygon_county).add_to(map_county)

# iterate through shops to place points on map, popup marker for shop name
for row_index, row_values in shops_county.iterrows():
    loc = [row_values['latitude'], row_values['longitude']]
    pop = str(row_values['name'])
    marker = folium.Marker(
        location = loc, 
        popup = pop) 
    
    marker.add_to(map_county)
map_county.save('../maps/map_nashville.html')

map_county

In [ ]:
# this could be useful to display information at country level - markers would be clustered showing how many shops
# in a county, then can have county-level maps of the highest concentration ready to go

# mapping shops with folium
# using cluster markers

# defining variables for map
area_center = ['39.50', '-98.35']
shops_country = shops_by_county

# create map centered on area
clustermap_country = folium.Map(location = area_center, zoom_start = 4)

#create a marker cluster
marker_cluster = MarkerCluster().add_to(clustermap_country)

# add county polygon to map
#folium.GeoJson(polygon_county).add_to(clustermap_county)

# iterate through shops to place points on map, popup marker for shop name
for row_index, row_values in shops_by_county.iterrows():
    loc = [row_values['latitude'], row_values['longitude']]
    pop = str(row_values[['name', 'address', 'city']])
    marker = folium.Marker(
        location = loc, 
        popup = pop) 
    
    marker.add_to(marker_cluster)

clustermap_country.save('../maps/mapcountry.html')

clustermap_country

In [ ]:
# save shops_by_county dataframe to csv

shops_by_county.to_csv('../data/df_shops_by_county.csv', index = False)